In [ ]:
import duckdb


In [ ]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [ ]:
df = con.execute("""
                SELECT *
                FROM (
                    SELECT *, ROW_NUMBER() OVER(PARTITION BY NATBR ORDER BY data_ingestao DESC) AS row
                    FROM bronze_z0019JB
                    WHERE data_ingestao >= '2025-03-30'
                ) AS t
                WHERE row = 1
                """).fetchdf()
df.head()

In [ ]:
df_final = df.drop(columns=['nome_arquivo', 'data_ingestao', 'row'])
df_final = df_final.rename(columns={"NATBR": "id"})
df_final = df_final.rename(columns={"MAKTX":"nm_produto"})
df_final = df_final.rename(columns={"WERKS":"id_categoria"})
df_final = df_final.rename(columns={"MAINS":"id_fornecedor"})
df_final = df_final.rename(columns={"LABST": "vl_preco"})
df_final.head(10)
#df_final.dtypes

In [ ]:
df_final.dtypes

In [ ]:
df2 = df_final
df2 = df2.astype({"id_categoria": "string"})
df2 = df2.astype({"id_fornecedor": "int32"})
df2 = df2.astype({"vl_preco": "float64"})
df2 = df2.astype({"nm_produto": "string"})
df2 = df2.astype({"id": "int32"})
df2.head(10)

In [ ]:

# Verificar as colunas existentes no DataFrame
print("Colunas disponíveis no DataFrame:", df2.columns)

# Verificar se a coluna 'id' existe
if 'id' in df2.columns:
    # Alterar o tipo da coluna se ela existir
    df2 = df2.astype({"id": int})
else:
    print("A coluna 'id' não existe no DataFrame.")
    # Opcional: Criar a coluna 'id' se necessário
    # df2['id'] = 0  # Exemplo: criar a coluna com valores padrão


#df2.dtypes


In [ ]:
con.execute("""
CREATE TABLE IF NOT EXISTS produtos (
    id BIGINT,
    nm_produto TEXT,
    id_categoria TEXT,
    id_fornecedor BIGINT,
    vl_preco FLOAT
)
""")


In [ ]:
con.execute("INSERT INTO produtos SELECT * FROM df2")

In [ ]:
df_resultado = con.execute("SELECT * FROM produtos").fetchdf()
df_resultado.head(10)

In [ ]:
con.close()